# Fáza 2 IAU - Daniela Paľuchová 50%, Mark Bartalos 50% 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from datetime import datetime
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import PowerTransformer
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import OneHotEncoder


In [ ]:
products = pd.read_csv("data/product.csv", sep='\t')
sessions = pd.read_csv("data/session.csv", sep='\t')
users = pd.read_csv("data/user.csv", sep='\t')

Vyriešené chyby z predchádzajúcej fázy

In [ ]:
users = users.drop_duplicates(subset='user_id', keep='first')
users['race'] = users['race'].fillna('unspecified')
users['birthdate'] = pd.to_datetime(users['birthdate'], errors='coerce')
sessions['ack']=sessions['ack'].astype(int)
duplikatne_riadky = sessions[sessions.duplicated()]
sessions.drop_duplicates(inplace=True)
sessions['session_start'] = pd.to_datetime(sessions['session_start'], errors='coerce')

In [ ]:
products.head()

In [ ]:
products.info()

In [ ]:
sessions.head()

In [ ]:
sessions.info()

In [ ]:
users.head()

In [ ]:
users.info()

Po skúšaní spojiť session s product som prisšla k zisteniu, že product_ean sa nezhodujú, takže sa tie dva datasety nedajú spojiť. Nevidím žiadne logické riešenie ako by sa to dalo napraviť. Takže budem spájať a následne pracovať iba s users a sessions.

In [ ]:
merged_data = pd.merge(sessions, users, on='user_id', how='left')
merged_data.info()


*Ako prvé idem vymatať stlpce, ktoré sú mi zbytočné*</br>
Vyrobila som si stlpec age (nahradila som aj chýbajúce hodnoty priemerom) pomocou stlpca birthday, ktorý následne budem vymazávať

In [ ]:
merged_data['age'] = ((datetime.now() - merged_data['birthdate']).dt.days / 365.25)
merged_data['age'] = merged_data['age'].round()
mean_age = merged_data['age'].mean()
merged_data['age'].fillna(mean_age, inplace=True)
merged_data['age'] = merged_data['age'].round().astype(int)

merged_data.head()

In [ ]:
if 'username' in merged_data.columns:
    merged_data.drop('username', axis='columns', inplace=True)
    print("vymazanie username")
if 'residence' in merged_data.columns:
    merged_data.drop('residence', axis='columns', inplace=True)
    print("vymazanie residence")
if 'current_location' in merged_data.columns:
    merged_data.drop('current_location', axis='columns', inplace=True)
    print("vymazanie current_location")
if 'name' in merged_data.columns:
    merged_data.drop('name', axis='columns', inplace=True)
    print("vymazanie name")
if 'mail' in merged_data.columns:
    merged_data.drop('mail', axis='columns', inplace=True)
    print("vymazanie mail")
if 'address' in merged_data.columns:
    merged_data.drop('address', axis='columns', inplace=True)
    print("vymazanie address")
if 'birthdate' in merged_data.columns:
    merged_data.drop('birthdate', axis='columns', inplace=True)
    print("vymazanie birthdate")
if 'session_start' in merged_data.columns:
    merged_data.drop('session_start', axis='columns', inplace=True)
    print("vymazanie session_start")
if 'sex' in merged_data.columns:
    merged_data.drop('sex', axis='columns', inplace=True)
    print("vymazanie sex")
merged_data.head()

Vymazali sme hodnoty, ktoré su pre každého usera jedinečné a teda nám nič nepovedia o ack, teda username, residence, current_location, name, mail, address, birthdate a session_start. Dalej sme vymazali aj atribút sex, pretože má okolo 60% NaN hodnot a analyzovať ack na základe tohto atribútu by teda nebolo presné._start

In [ ]:
merged_data['race'] = merged_data['race'].fillna('unspecified')
merged_data['job'] = merged_data['job'].fillna('unspecified')
merged_data.head()

Doplnenie chýbajúcich hodnôt do objektových atribútov</br>
Race má už v pôvodných dátach hodnotu 'unspecified' a v tomto kontexte je to isté ako NaN, tak všetky NaN hodnoty tým nahradíme. V atribúte jobs sa nachádza veľké množstvo NaN hodnôt a veľa rozličných druhov prác, takže by nebolo logické to nahradzovať napríklad pomerom prác, takže použijeme znova označenie 'unspecified'. Ziaden pocet NaN v spomýnaných atribútov vyššie nebol natoľko nízky, aby sme si ich mohli dovoliť vymazať. Ďalšie objektové atribúty nemajú NaN hodnoty 

In [ ]:
ce_ordinal = ce.OrdinalEncoder(cols=['race'])
merged_data= ce_ordinal.fit_transform(merged_data)
ce_ordinal = ce.OrdinalEncoder(cols=['job'])
merged_data= ce_ordinal.fit_transform(merged_data)
ce_ordinal = ce.OrdinalEncoder(cols=['browser_name'])
merged_data= ce_ordinal.fit_transform(merged_data)
ce_ordinal = ce.OrdinalEncoder(cols=['screen_height'])
merged_data= ce_ordinal.fit_transform(merged_data)
ce_ordinal = ce.OrdinalEncoder(cols=['screen_width'])
merged_data= ce_ordinal.fit_transform(merged_data)
merged_data.head()

Následne objectové atribúty zmeníme na číselne pomocou Ordinal Encoding

In [ ]:
merged_data.info()

In [ ]:
def deleteNaN(merged_data):
    for column in merged_data.columns:
        number_NaN=merged_data[column].isna().sum()
        if 0<number_NaN:
            merged_data = merged_data.dropna(subset=[column])
    return merged_data

In [ ]:
def replaceNaNmean(merged_data):
    for column in merged_data.columns:
        number_NaN=merged_data[column].isna().sum()
        if 0<number_NaN:
           merged_data[column].fillna(merged_data[column].mean(), inplace=True)
    return merged_data

In [ ]:
def replaceNaNmedian(merged_data):
    for column in merged_data.columns:
        number_NaN=merged_data[column].isna().sum()
        if 0<number_NaN:
           merged_data[column].fillna(merged_data[column].median(), inplace=True)
    return merged_data

In [ ]:
#merged_data=deleteNaN(merged_data)
#merged_data=replaceNaNmean(merged_data)
merged_data=replaceNaNmedian(merged_data)

Vytvorili sme funkcie na vyriešenie NaN hodnôt. Môžeme si vybrať, či chceme ich vymazať, nahradiť priemerom alebo medianom. 

In [ ]:
merged_data.info()

In [ ]:
fig, ax = plt.subplots(figsize=(30, 20))
sns.boxplot(data=merged_data[['pct_wild_mouse', 'pct_input', 'pct_scrandom', 'pct_scroll_move', 'pct_click_product_info', 'wild_mouse_duration', 'total_load_time', 'pct_click']])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(30, 20))
sns.boxplot(data=merged_data[['pct_doubleclick', 'pct_mouse_move', 'pct_scroll_move_duration', 'scroll_move_total_rel_distance', 'page_activity_duration', 'mouse_move_total_rel_distance', 'pct_rage_click', 'pct_mouse_click']])
plt.show()

Na grafoch môžeme vidieť, že atribúty majú outlierov. Niektoré viac, niektoré menej. V grafe som obsiahla všetky atribúty, ktorých atribúty by bolo potrebné ošetriť na outlierov (ešte session duration, ale to som nedávala do grafu, lebo má veľké hodnoty oproti ostatným atribútom a nie je dobre vidieť potom ostatné)


In [ ]:
#Funkcia prevzatá zo vzorových notebookov
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

In [ ]:
def delete_outliers(merged_data):
    count=0
    pocet_zaznamov_na_zaciatku=merged_data.shape[0]
    for column in merged_data[['pct_doubleclick', 'pct_mouse_move', 'pct_scroll_move_duration', 'scroll_move_total_rel_distance', 'page_activity_duration', 'mouse_move_total_rel_distance', 'pct_rage_click', 'pct_mouse_click', 'pct_wild_mouse', 'pct_input', 'pct_scrandom', 'pct_scroll_move', 'pct_click_product_info', 'wild_mouse_duration', 'total_load_time', 'pct_click', 'session_duration']]:
        outliers = identify_outliers(merged_data[column])
        count+=outliers.count()
        merged_data=merged_data.drop(outliers.index)

    print(f'Počet vychýlených hodnôt je {count}')
    print(f'Percento zaznamov, ktore ostali v percentach: {merged_data.shape[0]*100/pocet_zaznamov_na_zaciatku}')
    return merged_data

In [ ]:
merged_data.info()

In [ ]:

merged_data = delete_outliers(merged_data)
merged_data.info()

Vymazanie outlierov, zistili sme, že skoro 20% dát sme stratili

In [ ]:
def delete_outliers_quantile(merged_data):
    count=0
    for column in merged_data[['pct_doubleclick', 'pct_mouse_move', 'pct_scroll_move_duration', 'scroll_move_total_rel_distance', 'page_activity_duration', 'mouse_move_total_rel_distance', 'pct_rage_click', 'pct_mouse_click', 'pct_wild_mouse', 'pct_input', 'pct_scrandom', 'pct_scroll_move', 'pct_click_product_info', 'wild_mouse_duration', 'total_load_time', 'pct_click', 'session_duration']]:
        outliers = identify_outliers(merged_data[column])
        lower_q = merged_data[column].quantile(0.05)
        uper_q = merged_data[column].quantile(0.95)
        count+=outliers.count()
        merged_data[column] = merged_data[column].clip(lower_q, uper_q)
        
    print(f'Počet zmenenych hodnôt je {count}')
    return merged_data



In [ ]:
#merged_data = delete_outliers_quantile(merged_data)
#merged_data.info()

Alternatíva pre outlierov je, že ich nahradím hraničnými hodnotami, teda quantilom 0.05 alebo 0.95

# 2. Predspracovanie dat

Údaje rozdelíme do 2 rôznych súborov údajov, jedna bude určená na trénovanie a druhá časť bude slúžiť na testovanie predpokladaných výsledkov. Tréningová časť bude tvoriť 80 % údajov a testovacia časť bude tvoriť 20 % kompletných údajov.

In [ ]:
features = merged_data.drop('ack', axis=1)
target = merged_data['ack']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

X_train

### Encoding

Všetky užitočné textové stĺpce boli zakódované v predchádzajúcej časti pomocou ordinálneho kódovania, kde boli rôzne kategórie textu zakódované do čísel, pričom každej kategórii bolo priradené jedinečné číslo.

### Škálovanie


Údaje normalizujeme pomocou MinMaxScaler. Používame MinMaxScaler, pretože tento scaler poskytuje rovnomerné škálovanie a zaisťuje, že žiadna jediná funkcia nedominuje modelu kvôli jeho mierke. Tiež zachováva vzťahy medzi pôvodnými dátovými bodmi.

In [ ]:
scaler = MinMaxScaler()
X_train.hist(figsize=(25,25), bins=50)
X_scaled = scaler.fit_transform(X_train) 
X_scaled = pd.DataFrame(X_scaled, columns=X_train.columns)

Na týchto histogramoch môžeme vidieť, ako sú jednotlivé údaje distribuované pred škálovaním.

In [ ]:
X_scaled.hist(figsize=(25,25), bins=50)

Vidíme, že pri použití MinMaxScaler distribúcia dát nezmenila len ich veľkosť.

### Štandardné škálovanie

Používame štandardné škálovanie, aby sa údaje viac podobali Gaussovcom, ale extrémne hodnoty budú stále pretrvávať, čo bude musieť byť opravené v ďalšom kroku.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_scaled)
X_scaled = pd.DataFrame(X_scaled, columns=X_train.columns)
X_scaled.hist(figsize=(25,25), bins=50)

Po použití StandardScaling vidíme graf, že priemer údajov je viac vycentrovaný, ale distribúcia je stále zachovaná.

### Normálizovanie distribúcia údajov

Normalizácia pomocou PowerTransformer zaisťuje, že transformované údaje sú následne škálované tak, aby mali nulový priemer a jednotkový rozptyl a tiež aby boli naše údaje viac gaussovské.

In [ ]:
power = PowerTransformer(method='yeo-johnson', standardize=True) 
X_new = power.fit_transform(X_scaled)
X_new = pd.DataFrame(X_new, columns=X_train.columns)
X_new.hist(figsize=(25,25), bins=50)

Na grafe vidíme, že na strane sú veľké extrémne hodnoty, chceme naše dáta zjednotiť pomocou PowerTransformaer a yeo-johnsonovej metódy. Vďaka tomu bude distribúcia našich údajov gaussovskejšia. Ako vidíme, takmer všetky naše údaje začínajú vyzerať ako gaussovská krivka.

### Quantile transformer

Pomocou kvantilového transformátora transformujeme vlastnosti tak, aby sledovali rovnomernejšie a normálnejšie rozloženie.

In [ ]:
quantile = QuantileTransformer(output_distribution='normal')
X_new = quantile.fit_transform(X_new)
X_new = pd.DataFrame(X_new, columns=X_train.columns)
X_new.hist(figsize=(25,25), bins=50)

Ako vidíme na histogramoch, takmer všetky údaje sú ako gaussovské, to znamená, že existuje malý alebo žiadny rozdiel v distribúcii každého prvku.

### Feature selection

In [ ]:
transform = SelectKBest(mutual_info_regression, k=10)
transform.fit_transform( X_new, y_train)
result=pd.DataFrame({'column_name':X_train.columns, 'column_score':transform.scores_})
result.sort_values('column_score', ascending=False).head(10)

Podľa výsledkov môžeme vidieť, že prvé 3 atribúty majú omnoho vyššie skore ako ostatné, štvrtý má tiež vyššie oproti ostatným, ale porovnateľne nižsie ako spomínané 3 atribúty

In [ ]:
transform2 = SelectKBest(f_regression, k=10)
transform2.fit_transform( X_new, y_train)
result2=pd.DataFrame({'column_name':X_train.columns, 'column_score':transform2.scores_})
result2.sort_values('column_score', ascending=False).head(10)

Podľa výsledkov môžeme vidieť, že prvé 3 atribúty majú omnoho vyššie skore ako ostatné, štvrtý má tiež vyššie oproti ostatným, ale porovnateľne nižsie ako spomínané 3 atribúty

In [ ]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit( X_new, y_train)
selector = SelectFromModel(estimator=lsvc, prefit=True)
pd.DataFrame({'attr':X_train.columns, 'score':lsvc.coef_[0]}).sort_values('score', ascending=False).head(10)

Podľa výsledkov môžeme vidieť, že prvé 2 atribúty majú omnoho vyššie skore ako ostatné, tretí, štvrtý a piaty má tiež vyššie oproti ostatným, ale porovnateľne nižsie ako spomínané 2 atribúty

In [ ]:
selector = SelectFromModel(estimator=LogisticRegression()).fit( X_new, y_train)
result3=pd.DataFrame({'column_name':X_train.columns, 'column_score':selector.estimator_.coef_[0]})
result3.sort_values('column_score', ascending=False).head(10)

Podľa výsledkov môžeme vidieť, že prvé 2 atribúty majú omnoho vyššie skore ako ostatné, tretí, prípadne štvrtý má tiež vyššie oproti ostatným, ale porovnateľne nižsie ako spomínané 2 atribúty

Z výsledkov môžeme vidieť, že najdôležitejšie atribúty sú pct_clic a mouse_move_total_rel_distance, kedže mali vo všetkých výsledkoch výrazne vyššie skore. Do úvahy môžem zobrať aj atribúť scroll_move_total_rel_distance, kedže tiež mal vyššie skore v vačšine testov. Poradie by som určila takéto:
1. pct_clic
2.  mouse_move_total_rel_distance
3.  scroll_move_total_rel_distance

### Predspracovanie pomocou pipelines

Aby sme zrefaktorovali naše spracovanie údajov, najskôr použijeme techniky čistenia údajov, ktoré sme robili predtým. Tieto kroky boli zhrnuté vo funkcii data_correction. Tu odstránime nepotrebné duplikáty používateľov a sessions, vypočítame vek používateľov, kódovanie stĺpcov a vynechanie nepotrebných stĺpcov.

Vytvorenie pipelinu pozostáva zo štyroch krokov:
- Handlovanie outlierov:<br>
Táto funkcia umožňuje rôzne stratégie na správu odľahlých hodnôt v množine údajov.
  - Ak je outlier_handling nastavené na 'remove', používa metódu Local Outlier Factor na identifikáciu a odstránenie odľahlých hodnôt z číselných stĺpcov.
  - Ak je nastavené na 'quantile', použije sa vlastná metóda delete_outliers_quantile (nie je definovaná v úryvku), pravdepodobne na spracovanie odľahlých hodnôt na základe kvantilových prahov.
  - Možnosť 'keep' ponecháva odľahlé hodnoty nedotknuté.
<br>
- Konfigurácia imputera:<br>
Funkcia skontroluje platné hodnoty imputer_type ('priemer', 'medián' alebo 'najčastejšie') a vyvolá chybu pre neplatné vstupy.
Konfiguruje imputer na doplnenie chýbajúcich hodnôt podľa zadanej stratégie.

- Kódovanie kategorických stĺpcov:<br>
Ak má hodnota encode_categorical_columns hodnotu True, funkcia použije jednorazové kódovanie na kategorické stĺpce.
Toto je ďalej spresnené, ak je špecifikovaný only_columns, čím sa zabezpečí, že kódovanie sa použije iba na určené stĺpce.

- Configuracia pipelinu: <br>
Funkcia vytvára objekt Pipeline, pričom postupne pridáva kroky pre jednorazové kódovanie (podmienené), imputáciu, škálovanie (pomocou MinMaxScaler), štandardizáciu (pomocou StandardScaler),  PowerTransformer a kvantilovú transformáciu (pomocou QuantileTransformer).

In [ ]:
products = pd.read_csv("data/product.csv", sep='\t')
sessions = pd.read_csv("data/session.csv", sep='\t')
users = pd.read_csv("data/user.csv", sep='\t')

users = users.drop_duplicates(subset='user_id', keep='first')
users['race'] = users['race'].fillna('unspecified')
users['birthdate'] = pd.to_datetime(users['birthdate'], errors='coerce')
sessions['ack']=sessions['ack'].astype(int)
duplikatne_riadky = sessions[sessions.duplicated()]
sessions.drop_duplicates(inplace=True)
sessions['session_start'] = pd.to_datetime(sessions['session_start'], errors='coerce')

merged_data = pd.merge(sessions, users, on='user_id', how='left')


def data_correction(merged_data):
    merged_data['age'] = ((datetime.now() - merged_data['birthdate']).dt.days / 365.25)
    merged_data['age'] = merged_data['age'].round()
    mean_age = merged_data['age'].mean()
    merged_data['age'].fillna(mean_age, inplace=True)
    merged_data['age'] = merged_data['age'].round().astype(int)

    if 'username' in merged_data.columns:
        merged_data.drop('username', axis='columns', inplace=True)
    if 'residence' in merged_data.columns:
        merged_data.drop('residence', axis='columns', inplace=True)
    if 'current_location' in merged_data.columns:
        merged_data.drop('current_location', axis='columns', inplace=True)
    if 'name' in merged_data.columns:
        merged_data.drop('name', axis='columns', inplace=True)
    if 'mail' in merged_data.columns:
        merged_data.drop('mail', axis='columns', inplace=True)
    if 'address' in merged_data.columns:
        merged_data.drop('address', axis='columns', inplace=True)
    if 'birthdate' in merged_data.columns:
        merged_data.drop('birthdate', axis='columns', inplace=True)
    if 'session_start' in merged_data.columns:
        merged_data.drop('session_start', axis='columns', inplace=True)
    if 'sex' in merged_data.columns:
        merged_data.drop('sex', axis='columns', inplace=True)

    merged_data['race'] = merged_data['race'].fillna('unspecified')
    merged_data['job'] = merged_data['job'].fillna('unspecified')
    merged_data=replaceNaNmedian(merged_data)

    ce_ordinal = ce.OrdinalEncoder(cols=['race'])
    merged_data= ce_ordinal.fit_transform(merged_data)
    ce_ordinal = ce.OrdinalEncoder(cols=['job'])
    merged_data= ce_ordinal.fit_transform(merged_data)
    ce_ordinal = ce.OrdinalEncoder(cols=['browser_name'])
    merged_data= ce_ordinal.fit_transform(merged_data)
    ce_ordinal = ce.OrdinalEncoder(cols=['screen_height'])
    merged_data= ce_ordinal.fit_transform(merged_data)
    ce_ordinal = ce.OrdinalEncoder(cols=['screen_width'])
    merged_data= ce_ordinal.fit_transform(merged_data)

    return merged_data

merged_data = data_correction(merged_data)

def create_preprocessor(outlier_handling, imputer_type, encode_categorical_columns, only_columns, data):
    
    # Outlier detection
    if outlier_handling == 'remove':
        lof = LocalOutlierFactor()
        outliers = lof.fit_predict(data.select_dtypes(include=['number']))
        data = data[outliers != -1]
        if only_columns is not None:
            data = data[outliers != -1]
            data = data[only_columns]
    elif outlier_handling == 'keep':
        pass
    elif outlier_handling == 'quantile':
        data = delete_outliers_quantile(data)

    # Imputer configuration
    if imputer_type not in ['mean', 'median', 'most_frequent']:
        raise ValueError("Invalid imputer_type value. Use 'mean', 'median', or 'most_frequent'.")

    pipeline = Pipeline(steps=[])

    if encode_categorical_columns:
        columns = data.select_dtypes(include=['object']).columns
        if only_columns is not None:
            for column in columns:
                if column not in only_columns:
                    columns = columns.drop(column)

        pipeline.steps.append(('onehot', ColumnTransformer([("onehot", OneHotEncoder(), make_column_selector(dtype_include='object'))], remainder='passthrough')))
      
    
    pipeline.steps.append(('imputer', SimpleImputer(strategy=imputer_type)))
    pipeline.steps.append(('scaler', MinMaxScaler()))
    pipeline.steps.append(('standard', StandardScaler()))
    pipeline.steps.append(('power', PowerTransformer(method='yeo-johnson', standardize=True)))
    pipeline.steps.append(('quantile', QuantileTransformer(output_distribution='normal')))

    return pipeline

features = merged_data.drop('ack', axis=1)
target = merged_data['ack']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

preprocessor = create_preprocessor('quantile', 'median', True, None, X_test)
X_test
X_encoded = preprocessor.fit_transform(X_test)
X_encoded


In [ ]:
X_train.to_csv('trenovaci.csv', index=False)